In [1]:
import os
from torchvision import datasets, transforms

# Define a transformation for the dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# Load an existing dataset
# You can also replace 'root' with a path to a local image dataset directory
dataset = datasets.FakeData(size=1000, image_size=(3, 224, 224), transform=transform)


In [2]:
import dask.bag as db
from PIL import Image
import numpy as np

# Define a function to simulate processing, like resizing or normalizing
def process_image(image_tensor):
    # Example: Convert tensor to PIL Image (assuming image_tensor is a tensor)
    image = transforms.ToPILImage()(image_tensor)
    image = image.resize((128, 128))  # Example resizing
    return np.array(image)

# Use Dask to process images in parallel
images = [dataset[i][0] for i in range(len(dataset))]  # Extract images
bag = db.from_sequence(images).map(process_image)
processed_images = bag.compute()


In [3]:
import torch
import torchvision.models as models

# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)
model.eval()

# Function to extract features
def extract_features(image_tensor):
    with torch.no_grad():
        features = model(image_tensor.unsqueeze(0))  # Add batch dimension
    return features.squeeze().numpy()

# Process images in parallel to extract features
features = [extract_features(image) for image in images]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 86.4MB/s]


In [4]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Convert features to a NumPy array for processing
features_array = np.array(features)

# Step 1: Use PCA to reduce dimensions to 50 (optional)
pca = PCA(n_components=50)
pca_features = pca.fit_transform(features_array)

# Step 2: Apply t-SNE for 2D visualization
tsne = TSNE(n_components=2, perplexity=30, random_state=0)
reduced_embeddings = tsne.fit_transform(pca_features)


In [6]:
!pip install dash
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 39.1 MB/s eta 0:00:00


In [7]:
from dash import Dash, html, dcc
import plotly.express as px

app = Dash(__name__)

# Create a scatter plot of the 2D embeddings
fig = px.scatter(
    x=reduced_embeddings[:, 0],
    y=reduced_embeddings[:, 1],
    title="2D Visualization of Image Embeddings"
)

# Define the layout of the Dash app
app.layout = html.Div([
    html.H1("Image Data Embeddings Visualization"),
    dcc.Graph(id='scatter-plot', figure=fig)
])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>